### Import Dependencies

In [7]:
import numpy as np 
import pandas as pd
import mediapipe as mp

from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

### Read Dataset 

In [8]:
data = pd.read_csv('../data.csv')
data.drop('Index', axis=1, inplace=True)
data.head()

,0,1,2,3,4,5,6,7,8,9,...,459,460,461,462,463,464,465,466,467,label
0,0.156041,0.034724,0.071444,0.078117,0.0,0.043656,0.147673,0.290654,0.227140,0.271585,...,0.063550,0.104668,0.057380,0.062866,0.189442,0.170856,0.159821,0.323322,0.348375,0
1,0.099105,0.033467,0.048210,0.096153,0.0,0.050459,0.181863,0.332776,0.276899,0.323902,...,0.061139,0.101462,0.051161,0.050882,0.230257,0.214284,0.201468,0.353529,0.377521,0
2,0.160259,0.033538,0.105772,0.086060,0.0,0.046421,0.181095,0.224147,0.273403,0.312137,...,0.071666,0.147298,0.067166,0.087085,0.274896,0.251641,0.229446,0.382837,0.408696,0
3,0.139705,0.035719,0.060722,0.098022,0.0,0.051891,0.187796,0.330197,0.296981,0.350095,...,0.063510,0.106100,0.055346,0.057549,0.232088,0.215398,0.203230,0.354787,0.383226,0
4,0.144930,0.030739,0.065995,0.082237,0.0,0.041594,0.145923,0.312308,0.210267,0.247076,...,0.035856,0.051338,0.042271,0.047047,0.154600,0.146428,0.139789,0.194182,0.204608,0


### Split variables from label

In [9]:
x = data.iloc[:, :-1]
y = data.iloc[:, -1]


In [10]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

### Initial Model

In [11]:
inputs = keras.Input(shape = 468)

z = layers.Dense(128, activation="relu")(inputs)
z = layers.Dense(128, activation="relu")(z)
outputs = layers.Dense(6, activation="softmax")(z)

model = keras.Model(inputs, outputs)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 468)]             0         
                                                                 
 dense_3 (Dense)             (None, 128)               60032     
                                                                 
 dense_4 (Dense)             (None, 128)               16512     
                                                                 
 dense_5 (Dense)             (None, 6)                 774       
                                                                 
Total params: 77,318
Trainable params: 77,318
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")],
)

# Train the model for 1 epoch from Numpy data
batch_size = 64
history = model.fit(x_train, y_train, batch_size=batch_size, epochs=100)

Epoch 1/100
407/407 [==============================] - 1s 2ms/step - loss: 1.6992 - acc: 0.2969
Epoch 2/100
407/407 [==============================] - 1s 1ms/step - loss: 1.5523 - acc: 0.3855
Epoch 3/100
407/407 [==============================] - 1s 2ms/step - loss: 1.4582 - acc: 0.4314
Epoch 4/100
407/407 [==============================] - 1s 1ms/step - loss: 1.4429 - acc: 0.4340
Epoch 5/100
407/407 [==============================] - 1s 1ms/step - loss: 1.4005 - acc: 0.4501
Epoch 6/100
407/407 [==============================] - 1s 1ms/step - loss: 1.3889 - acc: 0.4548
Epoch 7/100
407/407 [==============================] - 1s 2ms/step - loss: 1.3717 - acc: 0.4608
Epoch 8/100
407/407 [==============================] - 1s 2ms/step - loss: 1.3654 - acc: 0.4627
Epoch 9/100
407/407 [==============================] - 1s 2ms/step - loss: 1.3562 - acc: 0.4650
Epoch 10/100
407/407 [==============================] - 2s 4ms/step - loss: 1.3400 - acc: 0.4716
Epoch 11/100
407/407 [=================

In [13]:
predictions = model.predict(x_test)
predictions = keras.backend.argmax(
    predictions,
    axis=-1
)


In [14]:
accuracy_score(y_test, predictions)

0.5163076923076924